### Build Yield Curve

In [1]:
from QuantLib import *
import math
from pandas import DataFrame

In [2]:
calc_date = Date(30, 11, 2015)
Settings.instance().evaluationDate = calc_date

day_count = ActualActual()
calendar = UnitedStates()
business_convention = Following
end_of_month = False
settlement_days = 0
face_amount = 100
coupon_frequency = Period(Semiannual)

In [3]:
prices = [99.9935, 99.9576, 99.8119, 99.5472, 99.8867, 100.0664, 99.8711, 100.0547, 100.3047, 100.2266]

coupon_rates = [0.0, 0.0, 0.0, 0.0, 0.00875, 0.0125, 0.01625, 0.02, 0.0225, 0.03]

maturity_dates = [Date(24, 12, 2015), Date(25, 2, 2016),
                  Date(26, 5, 2016), Date(10, 11, 2016),
                  Date(30, 11, 2017), Date(15, 11, 2018),
                  Date(30, 11, 2020), Date(30, 11, 2022),
                  Date(15, 11, 2025), Date(15, 11, 2045)]

issue_dates =    [Date(25, 6, 2015), Date(27, 8, 2015),
                  Date(28, 5, 2015), Date(12, 11, 2015),
                  Date(30, 11, 2015), Date(16, 11, 2015),
                  Date(30, 11, 2015), Date(30, 11, 2015),
                  Date(16, 11, 2015), Date(15, 11, 2015)]

coupon_frequency = Period(6, Months)

In [4]:
bond_helpers = []

for coupon, issue_date, maturity_date, price in zip(coupon_rates, issue_dates, maturity_dates, prices):
    schedule = Schedule(calc_date,
                        maturity_date,
                        coupon_frequency,
                        calendar,
                        business_convention,
                        business_convention,
                        DateGeneration.Backward,
                        False)
    
    helper = FixedRateBondHelper(QuoteHandle(SimpleQuote(price)),
                                 settlement_days,
                                 face_amount,
                                 schedule,
                                 [coupon],
                                 day_count,
                                 business_convention)
    bond_helpers.append(helper)

In [5]:
yield_curve = PiecewiseCubicZero(calc_date, bond_helpers, day_count)
yield_curve_handle = YieldTermStructureHandle(yield_curve)
discount_factors = [yield_curve.discount(d) for d in maturity_dates]

DataFrame(zip(maturity_dates, discount_factors),
          columns=['Dates', 'Discount Factors'],
          index=[''] * len(maturity_dates))

,Dates,Discount Factors
,"December 24th, 2015",0.999935
,"February 25th, 2016",0.999576
,"May 26th, 2016",0.998119
,"November 10th, 2016",0.995472
,"November 30th, 2017",0.981524
,"November 15th, 2018",0.964278
,"November 30th, 2020",0.920306
,"November 30th, 2022",0.868533
,"November 15th, 2025",0.799447
,"November 15th, 2045",0.384829


Treasury futures contract, TYZ5  
treasury futures on the 10 year note for delivery in December 2015  
10-year 6% coupon note issued as of the calculation date  
Futures price for the TYZ5 is 127.0625

In [6]:
def create_tsy_security(bond_issue_date, bond_maturity_date, coupon_rate, coupon_frequency=Period(6, Months), 
                        day_count = ActualActual(), calendar = UnitedStates()):
    face_value = 100.0
    settlement_days = 0
    
    schedule = Schedule(bond_issue_date,
                        bond_maturity_date,
                        coupon_frequency,
                        calendar,
                        ModifiedFollowing,
                        ModifiedFollowing,
                        DateGeneration.Forward,
                        False)
    
    security = FixedRateBond(settlement_days,
                             face_value,
                             schedule,
                             [coupon_rate],
                             day_count)
    return security

In [7]:
bond_issue_date = calc_date
delivery_date = Date(1, 12, 2015)

bond_maturity_date = bond_issue_date + Period(10, Years)
day_count = ActualActual()
coupon_frequency = Period(6, Months)
coupon_rate = 6/100

deliverable = create_tsy_security(bond_issue_date,
                                  bond_maturity_date,
                                  coupon_rate,
                                  coupon_frequency,
                                  day_count,
                                  calendar)

bond_engine = DiscountingBondEngine(yield_curve_handle)
deliverable.setPricingEngine(bond_engine)

### Calculate the Z-Spread

Z-Spread is the static spread added to the yield curve to mathc the price of the security  
For a treasury security, you would expect this to be zero

In [8]:
futures_price = 127.0625
clean_price = futures_price * yield_curve.discount(delivery_date)

print(clean_price)

127.06215586167724


In [9]:
zspread = BondFunctions.zSpread(deliverable, clean_price, yield_curve, day_count,
                                Compounded, Semiannual, calc_date) * 10000

print("Z-Spread is %3.0fbp" %(zspread))

Z-Spread is  71bp


### Cheapest to Deliver

The seller of the futures contract, has to buy the delivery security from the market and sell it at an adjusted futures price  

#### Adjusted Futures Price = Futures Price * Conversion Factor

The gain or loss to seller is given by the basis  

#### Basis = Cash Price - Adjusted Futures Price

The cheapes to deliver is expected to be the security with the lowest basis

In [10]:
day_count = ActualActual()
basket = [(1.625, Date(15, 8, 2022), 97.921875),
          (1.625, Date(15, 11, 2022), 97.671875),
          (1.75, Date(30, 9, 2022), 98.546875),
          (1.75, Date(15, 5, 2023), 97.984375),
          (1.875, Date(31, 8, 2022), 99.375),
          (1.875, Date(31, 10, 2022), 99.296875),
          (2.0, Date(31, 7, 2022), 100.265625),
          (2.0, Date(15, 2, 2023), 100.0625),
          (2.0, Date(15, 2, 2025), 98.296875),
          (2.0, Date(15, 8, 2025), 98.09375),
          (2.125, Date(30, 6, 2022), 101.0625),
          (2.125, Date(15, 5, 2025), 99.25),
          (2.25, Date(15, 11, 2024), 100.546875),
          (2.25, Date(15, 11, 2025), 100.375),
          (2.375, Date(15, 8, 2024), 101.671875),
          (2.5, Date(15, 8, 2023), 103.25),
          (2.5, Date(15, 5, 2024), 102.796875),
          (2.75, Date(15, 11, 2023), 105.0625),
          (2.75, Date(15, 2, 2024), 104.875)]

securities = []
min_basis = 100; min_basis_index = -1

for i, b in enumerate(basket):
    coupon, maturity, price = b
    #print(i)
    #print(b)
    issue = maturity - Period(10, Years)
    s = create_tsy_security(issue, maturity, coupon/100)
    bond_engine = DiscountingBondEngine(yield_curve_handle)
    s.setPricingEngine(bond_engine)
    
    # conversion factor
    cf = BondFunctions.cleanPrice(s, 0.06, day_count, Compounded, Semiannual, calc_date)/100
    adjusted_futures_price = futures_price * cf
    basis = price - adjusted_futures_price
    
    if basis < min_basis:
        min_basis = basis
        min_basis_index = i
    securities.append((s, cf))

ctd_info = basket[min_basis_index]
ctd_bond, ctd_cf = securities[min_basis_index]
ctd_price = ctd_info[2]

print("%-30s = %lf" %("Minimum Basis", min_basis))
print("%-30s = %lf" %("Conversion Factor", ctd_cf))
print("%-30s = %lf" %("Coupon", ctd_info[0]))
print("%-30s = %s" %("Maturity", ctd_info[1]))
print("%-30s = %lf" %("Price", ctd_info[2]))
print(ctd_bond)

Minimum Basis                  = 0.450601
Conversion Factor              = 0.791830
Coupon                         = 2.125000
Maturity                       = June 30th, 2022
Price                          = 101.062500
<QuantLib.QuantLib.FixedRateBond; proxy of <Swig Object of type 'FixedRateBondPtr *' at 0x00000216B9FB7600> >


#### The basis is the loss for anotional of 100 that the seller accrues to close this contract

In [11]:
futures_maturity_date = Date(21, 12, 2015)
futures = FixedRateBondForward(calc_date, futures_maturity_date, Position.Long,
                               0.0, settlement_days, day_count,
                               calendar, business_convention, ctd_bond, yield_curve_handle, yield_curve_handle)

In [15]:
model_futures_price = futures.cleanForwardPrice()/ ctd_cf
implied_yield = futures.impliedYield(ctd_price/ctd_cf, futures_price, calc_date, Compounded, day_count).rate()
z_spread = BondFunctions.zSpread(ctd_bond, ctd_price, yield_curve, day_count, Compounded, Semiannual, calc_date)
# ytm = BondFunctions.yield(ctd_bond, ctd_price, day_count, Compounded, Semiannual, calc_date)

print("%-30s = %lf" %("Model Futures Price", model_futures_price))
print("%-30s = %lf" %("Market Futures Price", futures_price))
print("%-30s = %lf" %("Model Adjustment", model_futures_price - futures_price))
print("%-30s = %2.3f%%" %("Implied Yield", implied_yield * 100))
print("%-30s = %2.1fbps" %("Forward Z-Spread", z_spread*10000))

Model Futures Price            = 127.610365
Market Futures Price           = 127.062500
Model Adjustment               = 0.547865
Implied Yield                  = -7.473%
Forward Z-Spread               = 1.6bps
